# FINAL PROJECT: tourism destination in Europe

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import requests

### Creation df with API

In [172]:
url = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/tour_dem_tttot?lang=EN"

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print(data.keys())
    display(pd.json_normalize(data['extension']).T)

else:
    print(f"Erreur {response.status_code} : Impossible to retrieve data")

dict_keys(['version', 'class', 'label', 'source', 'updated', 'value', 'status', 'id', 'size', 'dimension', 'extension'])


,0
lang,EN
id,TOUR_DEM_TTTOT
agencyId,ESTAT
version,1.0
annotation,"[{'type': 'CREATED', 'date': '2022-10-13T11:34..."
datastructure.id,TOUR_DEM_TTTOT
datastructure.agencyId,ESTAT
datastructure.version,39.0
status.label.b,break in time series
status.label.e,estimated


In [17]:
import pandas as pd
import itertools

def eurostat_to_dataframe(data):
    # Extract dimensions and their values
    dimensions = data['dimension']
    dim_keys = list(dimensions.keys())
    dim_values = [dimensions[key]['category']['index'].keys() for key in dim_keys]
    
    # Create a MultiIndex from the Cartesian product of dimension values
    index = pd.MultiIndex.from_product(dim_values, names=dim_keys)

    if isinstance(data['value'], dict):
        values_list = [data['value'].get(str(i), None) for i in range(len(index))]
    else:
        values_list = data['value']
    
    # Create the DataFrame
    df = pd.DataFrame({'value': values_list}, index=index)
    
    # Reset the index to convert MultiIndex to columns
    df = df.reset_index()
    
    return df

# Use the function
df = eurostat_to_dataframe(data)

# Display the first few rows of the DataFrame
print(df.head())

# Basic info about the DataFrame
print(df.info())

  freq c_dest purpose duration unit        geo  time        value
0    A    DOM   TOTAL    N_GE1   NR  EU27_2020  2012  818977006.0
1    A    DOM   TOTAL    N_GE1   NR  EU27_2020  2013  795564577.0
2    A    DOM   TOTAL    N_GE1   NR  EU27_2020  2014  780507241.0
3    A    DOM   TOTAL    N_GE1   NR  EU27_2020  2015  769598755.0
4    A    DOM   TOTAL    N_GE1   NR  EU27_2020  2016  791264523.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   freq      10692 non-null  object 
 1   c_dest    10692 non-null  object 
 2   purpose   10692 non-null  object 
 3   duration  10692 non-null  object 
 4   unit      10692 non-null  object 
 5   geo       10692 non-null  object 
 6   time      10692 non-null  object 
 7   value     9027 non-null   float64
dtypes: float64(1), object(7)
memory usage: 668.4+ KB
None


In [18]:
# Fetch the data
url = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/tour_dem_tttot?lang=EN"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()  # This is the 'data' we pass to the function
    
    # Use the function to create the DataFrame
    df = eurostat_to_dataframe(data)
    
    # Display the first few rows of the DataFrame
    print(df.head())
    
    # Basic info about the DataFrame
    print(df.info())
else:
    print(f"Error {response.status_code}: Unable to retrieve data")

  freq c_dest purpose duration unit        geo  time        value
0    A    DOM   TOTAL    N_GE1   NR  EU27_2020  2012  818977006.0
1    A    DOM   TOTAL    N_GE1   NR  EU27_2020  2013  795564577.0
2    A    DOM   TOTAL    N_GE1   NR  EU27_2020  2014  780507241.0
3    A    DOM   TOTAL    N_GE1   NR  EU27_2020  2015  769598755.0
4    A    DOM   TOTAL    N_GE1   NR  EU27_2020  2016  791264523.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   freq      10692 non-null  object 
 1   c_dest    10692 non-null  object 
 2   purpose   10692 non-null  object 
 3   duration  10692 non-null  object 
 4   unit      10692 non-null  object 
 5   geo       10692 non-null  object 
 6   time      10692 non-null  object 
 7   value     9027 non-null   float64
dtypes: float64(1), object(7)
memory usage: 668.4+ KB
None


In [20]:
df['value'].isna().sum()

1665

In [21]:
df.dropna(inplace=True)

In [25]:
df.duplicated().sum()

0

In [22]:
df.shape

(9027, 8)

In [26]:
df.head()

,freq,c_dest,purpose,duration,unit,geo,time,value
0,A,DOM,TOTAL,N_GE1,NR,EU27_2020,2012,818977006.0
1,A,DOM,TOTAL,N_GE1,NR,EU27_2020,2013,795564577.0
2,A,DOM,TOTAL,N_GE1,NR,EU27_2020,2014,780507241.0
3,A,DOM,TOTAL,N_GE1,NR,EU27_2020,2015,769598755.0
4,A,DOM,TOTAL,N_GE1,NR,EU27_2020,2016,791264523.0


### DF destinations

In [32]:
df1 = pd.read_csv("../data/raw/destinations.csv", encoding='latin-1')
df1.head()

,Destination,Region,Country,Category,Latitude,Longitude,Approximate Annual Tourists,Currency,Majority Religion,Famous Foods,Language,Best Time to Visit,Cost of Living,Safety,Cultural Significance,Description
0,Rome,Lazio,Italy,City,41.902782,12.496366,14 million,Euro,Roman Catholic,"Pizza, Pasta, Gelato",Italian,Spring (April-May) or Fall (Sept-Oct),Medium-high,"Generally safe, but watch out for pickpockets","The capital city, known for its historical lan...","A hub of ancient history and modern culture, w..."
1,Florence,Tuscany,Italy,City,43.769581,11.255772,10 million,Euro,Roman Catholic,"Pizza, Pasta, Gelato",Italian,Spring (April-May) or Fall (Sept-Oct),Medium-high,"Generally safe, but watch out for pickpockets","A Renaissance city famous for its art, archite...","Home to world-class museums, including the Uff..."
2,Venice,Veneto,Italy,City,45.435559,12.336196,10 million,Euro,Roman Catholic,"Pizza, Pasta, Gelato",Italian,Spring (April-May) or Fall (Sept-Oct),Medium-high,"Generally safe, but watch out for pickpockets","A unique city built on canals, known for its g...","An iconic city of water, renowned for romantic..."
3,Milan,Lombardy,Italy,City,45.464643,9.188540,7 million,Euro,Roman Catholic,"Risotto, Ossobuco, Panettone",Italian,Spring (April-May) or Fall (Sept-Oct),High,"Generally safe, but watch out for pickpockets","A fashion capital known for its shopping, muse...","A modern city with an ancient soul, featuring ..."
4,Naples,Campania,Italy,City,40.851330,14.254720,5 million,Euro,Roman Catholic,"Pizza, Pasta, Cannoli",Italian,Spring (April-May) or Fall (Sept-Oct),Medium,"Generally safe, but be aware of pickpockets","A vibrant city known for its delicious food, h...","Famous for pizza, Pompeii, and proximity to th..."


In [35]:
df1['Description'] = df1['Description'].fillna('')

In [37]:
df1.isna().sum()

Destination                    0
Region                         0
Country                        0
Category                       0
Latitude                       0
Longitude                      0
Approximate Annual Tourists    0
Currency                       0
Majority Religion              0
Famous Foods                   0
Language                       0
Best Time to Visit             0
Cost of Living                 0
Safety                         0
Cultural Significance          0
Description                    0
dtype: int64

In [36]:
df1.duplicated().sum()

0

### DF tripadvisor european restaurant

In [101]:
df2 = pd.read_csv("../data/raw/tripadvisor_european_restaurants.csv")
df2.head()

/var/folders/7w/mr3l1rjn21bd3r_kvq3bgr1h0000gn/T/ipykernel_91688/246582737.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("../data/raw/tripadvisor_european_restaurants.csv")


,restaurant_link,restaurant_name,original_location,country,region,province,city,address,latitude,longitude,...,excellent,very_good,average,poor,terrible,food,service,value,atmosphere,keywords
0,g10001637-d10002227,Le 147,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"10 Maison Neuve, 87510 Saint-Jouvent France",45.961674,1.169131,...,2.0,0.0,0.0,0.0,0.0,4.0,4.5,4.0,NaN,NaN
1,g10001637-d14975787,Le Saint Jouvent,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"16 Place de l Eglise, 87510 Saint-Jouvent France",45.957040,1.205480,...,2.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,g10002858-d4586832,Au Bout du Pont,"[""Europe"", ""France"", ""Centre-Val de Loire"", ""B...",France,Centre-Val de Loire,Berry,Rivarennes,"2 rue des Dames, 36800 Rivarennes France",46.635895,1.386133,...,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,g10002986-d3510044,Le Relais de Naiade,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Co...",France,Nouvelle-Aquitaine,Correze,Lacelle,"9 avenue Porte de la Correze 19170, 19170 Lace...",45.642610,1.824460,...,1.0,0.0,0.0,0.0,0.0,4.5,4.5,4.5,NaN,NaN
4,g10022428-d9767191,Relais Du MontSeigne,"[""Europe"", ""France"", ""Occitanie"", ""Aveyron"", ""...",France,Occitanie,Aveyron,Saint-Laurent-de-Levezou,"route du Montseigne, 12620 Saint-Laurent-de-Le...",44.208860,2.960470,...,4.0,7.0,0.0,0.0,0.0,4.5,4.5,4.5,NaN,NaN


In [102]:
df2 = df2.drop(['restaurant_link', 'original_location', 'country', 'region', 'province', 'city', 'default_language','working_shifts_per_week', 'latitude', 
                'longitude', 'popularity_generic', 'open_hours_per_week', 'features', 'atmosphere','keywords', 'special_diets',
               'excellent', 'very_good', 'average', 'poor', 'terrible', 'reviews_count_in_default_language'], axis=1)

In [103]:
df2.fillna({
    'awards': 'Unknown',
    'claimed': 'Unknown',
    'popularity_detailed': 'Unknown',
    'top_tags': 'Unknown',
    'price_level': 'Unknown',
    'price_range': 'Unknown',
    'meals': 'Unknown',
    'cuisines': 'Unknown',
    'original_open_hours': 'Unknown',
    'open_days_per_week': 'Unknown',    
}, inplace=True)

In [104]:
df2.isna().sum()

restaurant_name             1
address                     0
claimed                     0
awards                      0
popularity_detailed         0
top_tags                    0
price_level                 0
price_range                 0
meals                       0
cuisines                    0
vegetarian_friendly         0
vegan_options               0
gluten_free                 0
original_open_hours         0
open_days_per_week          0
avg_rating              96636
total_reviews_count     52235
food                   484072
service                479110
value                  480705
dtype: int64

In [105]:
df2 = df2.dropna()

In [106]:
df2.isna().sum()

restaurant_name        0
address                0
claimed                0
awards                 0
popularity_detailed    0
top_tags               0
price_level            0
price_range            0
meals                  0
cuisines               0
vegetarian_friendly    0
vegan_options          0
gluten_free            0
original_open_hours    0
open_days_per_week     0
avg_rating             0
total_reviews_count    0
food                   0
service                0
value                  0
dtype: int64

In [107]:
df2.shape

(595793, 20)

## Export csv

In [109]:
df.to_csv('../data/clean/trip_eurostat_clean.csv', index=False)

In [110]:
df1.to_csv('../data/clean/destinations_clean.csv', index=False)

In [111]:
df2.to_csv('../data/clean/restaurants_tripadvisor_clean.csv', index=False)